
[Dataset](https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view)

In [1]:
import os
Root = "speech-emotion-recognition-ravdess-data"
os.chdir(Root)

In [2]:
ls

Actor_01/ Actor_04/ Actor_07/ Actor_10/ Actor_13/ Actor_16/ Actor_19/ Actor_22/
Actor_02/ Actor_05/ Actor_08/ Actor_11/ Actor_14/ Actor_17/ Actor_20/ Actor_23/
Actor_03/ Actor_06/ Actor_09/ Actor_12/ Actor_15/ Actor_18/ Actor_21/ Actor_24/


In [3]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [18]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))  # spectrum 
        result=np.array([])  # get three kinkds of features and concatenate them into columns
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [19]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [22]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]  # must read absolute path
    for file in glob.glob("/Users/anlly/Desktop/ucl/Final_Project/ELEC0054_Research_Project_23_24-SN23043574/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        # print(file)
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    print(np.array(x).shape)  # 768, 180
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [23]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

(768, 180)


In [25]:
x_train
print(x_train.shape)  # features: 180

(576, 180)


In [26]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [27]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [28]:
#Initialize the Multi Layer Perceptron Classifier
# one input, one hidden with 300 unit,  one output
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [29]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [30]:
#Predict for the test set
y_pred=model.predict(x_test)

In [31]:
y_pred

array(['calm', 'fearful', 'fearful', 'calm', 'happy', 'fearful', 'calm',
       'happy', 'disgust', 'calm', 'happy', 'fearful', 'disgust', 'calm',
       'happy', 'calm', 'fearful', 'happy', 'fearful', 'happy', 'happy',
       'happy', 'fearful', 'happy', 'fearful', 'happy', 'fearful', 'calm',
       'happy', 'happy', 'happy', 'happy', 'calm', 'fearful', 'calm',
       'calm', 'calm', 'happy', 'fearful', 'happy', 'happy', 'happy',
       'calm', 'happy', 'happy', 'calm', 'calm', 'happy', 'calm', 'happy',
       'happy', 'happy', 'fearful', 'happy', 'fearful', 'disgust', 'calm',
       'happy', 'calm', 'calm', 'happy', 'fearful', 'calm', 'fearful',
       'fearful', 'happy', 'disgust', 'calm', 'happy', 'fearful', 'happy',
       'calm', 'fearful', 'fearful', 'calm', 'disgust', 'calm', 'calm',
       'calm', 'fearful', 'calm', 'fearful', 'fearful', 'disgust', 'calm',
       'happy', 'calm', 'calm', 'calm', 'fearful', 'calm', 'happy',
       'happy', 'fearful', 'calm', 'calm', 'fearful', 

In [32]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 65.10%


In [33]:
from sklearn.metrics import accuracy_score, f1_score

In [34]:
f1_score(y_test, y_pred,average=None)

array([0.70175439, 0.4       , 0.65909091, 0.75      ])

In [35]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,calm,calm
1,fearful,fearful
2,disgust,fearful
3,disgust,calm
4,happy,happy
5,fearful,fearful
6,calm,calm
7,happy,happy
8,disgust,disgust
9,calm,calm


In [36]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [38]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/Users/anlly/Desktop/ucl/Final_Project/ELEC0054_Research_Project_23_24-SN23043574/speech-emotion-recognition-ravdess-data/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['happy'], dtype='<U7')

In [39]:
feature

array([[-6.93497009e+02,  5.00643921e+01,  5.71450591e-01,
         1.43299656e+01,  3.33636999e+00, -2.54071975e+00,
        -4.05790901e+00, -1.07119989e+01, -7.29413891e+00,
         1.74018884e+00, -4.19064283e+00,  1.95466173e+00,
        -5.24789381e+00,  2.78142977e+00, -3.16756725e+00,
        -3.40008283e+00, -2.37803221e+00, -5.68717599e-01,
        -6.47753334e+00, -1.24320555e+00, -2.80542517e+00,
        -5.43635798e+00, -4.46875244e-01, -3.63516617e+00,
        -2.98372626e+00, -5.63902617e-01, -1.65101945e+00,
        -5.55944741e-01, -3.41018438e+00, -2.24465466e+00,
        -3.13058877e+00, -2.70089960e+00, -1.88821304e+00,
        -5.54154396e-01, -3.96459866e+00, -2.13485217e+00,
        -3.94577074e+00, -1.62457860e+00, -2.03990722e+00,
        -3.62910843e+00,  6.33734703e-01,  6.48760557e-01,
         6.21744275e-01,  6.34553194e-01,  6.60345674e-01,
         6.60463989e-01,  7.00948954e-01,  7.31556237e-01,
         7.46921539e-01,  7.23420441e-01,  7.16119647e-0